In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import h5py
import os
import re
from scipy.interpolate import griddata
#from dedalus import public as de

In [2]:
def getproj(z, r_inner, r_outer):    
    if z < r_inner: 
        start_point = z*np.tan(np.arccos(z/r_inner)) 
    else:
        start_point = 0
    end_point = z*np.tan(np.arccos(z/r_outer))
    gridpoints = int(rscale.shape[0]/(r_outer-r_inner) * (end_point-start_point))
    
    z_desired = np.asarray(np.linspace( start_point, end_point, num = gridpoints, endpoint = False))
    y_coord = np.asarray([z for i in z_desired])
    inter_coord = np.asarray([z_desired, y_coord]).T
    
    phi_pad = np.append(phiscale[:], 2*np.pi)
    s_pad   = np.append(z_desired[:], end_point)
    s_mesh, phi_mesh = np.meshgrid(s_pad, phi_pad)
    
    return inter_coord, s_mesh, phi_mesh

In [7]:
def getplotdata(data, xykeys, datakeys):
    datapoints = np.asarray([dat.ravel() for dat in data])
#    datadic = [dict(zip(xykeys, datpts)) for datpts in datapoints]
    datadic = dict(zip(xykeys, datapoints.T))
#    datas = np.asarray([[dic[key] for key in datakeys] for dic in datadic])
    datas = np.asarray([datadic[key] for key in datakeys]).T
    return datas

In [4]:
def plotnsave(val, s_mesh, phi_mesh):
    fig, ax = plt.subplots(subplot_kw=dict(polar=True))
    pcm = ax.pcolormesh(phi_mesh, s_mesh, val, cmap='RdYlBu_r')
    plt.colorbar(pcm)
    plt.savefig(str(mydir)+'/%s_%04i.png' %("new2_plot".format(0), counter), dpi=150)
    plt.close(fig)

In [5]:
files = []
mydir = "data_Ek1e-6Ra300_L127N63_max2e-7"
for file in os.listdir(mydir):
    if file.endswith(".h5"):
        files.append(os.path.join(mydir, file))
files.sort(key=lambda f: int(re.sub('\D','', f)))

r_inner = 7/13.
r_outer = 20./13.

In [8]:
#    if data.ndim == 5: np.linalg.norm(data, axis=4)

counter = 0
z = 0.1*(r_outer-r_inner)+r_inner
for file in files:
    df = h5py.File(file,"r")
    temp = df['tasks']['T']
    data = np.array(temp)
    
    if counter == 0:
        rscale = temp.dims[3][0][:]
        thetascale = temp.dims[2][0][:]
        phiscale = temp.dims[1][0][:]
    
        inter_coord, s_mesh, phi_mesh = getproj(z, r_inner, r_outer)
    
        xygrid = np.asarray([[r*np.sin(theta), r*np.cos(theta)] for r in rscale for theta in thetascale])
        inbetween = np.asarray([xy for xy in xygrid if (z - 1/20) <= xy[1] <= (z + 1/20)])
    
        xykeys = list(map(str,xygrid))
        datakeys = list(map(str, inbetween))    
    
    
    for dats in data:
        datas = getplotdata(dats, xykeys, datakeys)

        grid_z0 = np.asarray([griddata(inbetween, dat, inter_coord, method='cubic') for dat in datas])

        plotnsave(grid_z0, s_mesh, phi_mesh)
        
        counter += 1   

KeyboardInterrupt: 